# Imports

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
print(16*7*7)

784


# Set Device

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters

In [4]:
input_channel = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epoch = 5

# :Load Data

In [5]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


# CNN

#### # Stride(1, 1) means move one step forward or down when we are doing the FEATURE MAPPING

In [6]:
class CNN(nn.Module):
    
    def __init__(self, input_channel=1, num_classes=10):
        super(CNN, self).__init__()
        
        # Stride(1, 1) means move one step forward or down when we are getting the maximum in the square
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3),stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        #reshape it before it goes to fc1
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        
        return x
        
        
        

# initialize a network

In [7]:
model = CNN().to(device)

# Loss & optimz

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Trian NetWork

In [9]:
for epoch in range(1):
    
    ix = 0 
    
    for images, labels in train_loader:
        # load to device
        images = images.to(device=device)
        labels = labels.to(device=device)
        
        # forward 
        scores_pred = model(images)
        loss = criterion(scores_pred, labels)

        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        ix += 1


# Accuracy

##### With the model.eval() and model.train() i think it will give you a bit better prediction !!! 👇👇

In [10]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Train data is runing.....')
    else:
        print('Test data is runing.....')
        
    num_corr = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            scores = model(x) # for every pic we'll get 10 predictions SO we need to get the max of the tensor.
            #len(scores[0]) --- 10
            #len(scores) 64 
            
            # get max
            # one pass will get us 64 tensors predictions then we will get the max of every tensor.
            a, predictions = scores.max(1)
            num_corr += (predictions == y).sum()
            num_samples += predictions.size(0)
        
            
        return float(num_corr) / float(num_samples) * 100
        
    model.train()       

    
train_acc = check_accuracy(train_loader, model)
test_acc = check_accuracy(test_loader, model) 

print("Train_accu: ", train_acc)
print("Test_accu: ", test_acc)

Train data is runing.....
Test data is runing.....
Train_accu:  96.70333333333333
Test_accu:  96.78


In [11]:
print(12*4*4)

192
